### reading data from order event contsiner


In [0]:
from pyspark.sql.functions import col, from_json
from pyspark.sql.types import StructType, StringType, StructField, IntegerType, DateType, TimestampType

# Read Avro files
df = spark.read.format('avro') \
    .option('ignoreExtension', 'true') \
    .option("recursiveFileLookup", "true") \
    .load('abfss://order-events@telcostoragelayer.dfs.core.windows.net/')

# Decode binary body
decoded_df = df.select(
    col("body").cast("string").alias("body_decoded")
)
# convert json data into data frame
json_schema=StructType([
    StructField("OrderID", StringType(), True),
    StructField("CustomerID", StringType(), True),
    StructField("PlanID", IntegerType(), True),
    StructField("OrderDate", StringType(), True),  # You can cast to DateType later
    StructField("Status", StringType(), True),
    StructField("Amount", IntegerType(), True)
])
data=decoded_df.select(from_json(col("body_decoded"),json_schema).alias("data")).select("data.*").withColumnRenamed('OrderID','order_id').withColumnRenamed('OrderDate','order_date').withColumnRenamed('PlanID','plan_id').withColumnRenamed('Status','status').withColumnRenamed('Amount','amount').withColumnRenamed('CustomerID','customer_id')
#display(data)
data=data.coalesce(1)
data.write.mode('overwrite').format('parquet').option('path','abfss://bronze@telcostoragelayer.dfs.core.windows.net/order/').save()

In [0]:
data.display()
